In [ ]:
import pandas as pd
import shap
shap.initjs()
import os
os.environ['KERAS_BACKEND']='tensorflow'
import numpy as np
import tensorflow as tf
from sklearn.linear_model import BayesianRidge, LinearRegression, ElasticNet
from sklearn import svm
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score  
import xgboost as xgb
from skopt import BayesSearchCV 
tf.config.experimental_run_functions_eagerly(True)
import time
from scipy import stats
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import palettable
from collections import OrderedDict
import json

In [ ]:
verson = 1
Y = [
      'LE',
      'LE_Male',
      'LE_Female',
      'LE_Old',
      'LE_Teen',
      'LE_White',
      'LE_Black',
      'LE_Latino',
      'LE_AIAN',
      'LE_API',
     ]
X_name = [
         'GDP_PC',
         'Income_PC',
         'Medical_R',
         'Temp_Mean',
         'PRCP_Mean',
         'PM25',
         'DEM',
         'GJ',
         'GE',
        ]
Y_name_IV = ['GE','GJ']
IV = pd.read_excel("\Dataset.xlsx")
for i in range(len(Y_name_IV)):
    X_train, X_test, Y_train, Y_test = train_test_split(IV.loc[:, X_name], IV.loc[:, Y_name_IV[i]], train_size=Bayes_divide, shuffle=True, random_state=42)
    bayes_cv_tuner = BayesSearchCV(
                                    estimator = xgb.XGBRegressor(tree_method = 'gpu_hist', gpu_id = 2),
                                    search_spaces = {
                                                    'learning_rate': (0.05, 0.25, 'log-uniform'),
                                                    'min_child_weight': (1, 20, 'uniform'),
                                                    'max_depth': (5, 25, 'uniform'),
                                                    'max_delta_step': ( 0, 20, 'uniform'),
                                                    'alpha':(0, 5, 'uniform'),
                                                    'lambda':(0, 5, 'uniform'),
                                                    },    
                                    n_iter = 1000,   
                                    verbose = 3,
                                    cv = 5
                                    )
    bayes_cv_tuner.fit(X_train, Y_train)
    info = json.loads(json.dumps(bayes_cv_tuner.best_params_))
    X_train, X_test, Y_train, Y_test = train_test_split(IV.loc[:, X_name], IV.loc[:, Y_name_IV[i]], train_size=0.7, shuffle=True, random_state=42)
    model_xgb = xgb.XGBRegressor(params=info, n_estimators = 1000)
    eval_set = [(X_test, Y_test)]
    model_xgb.fit(X_train, Y_train, early_stopping_rounds = 100, eval_metric="rmse", eval_set=eval_set, verbose=False)
    IV[Y_name_IV[i]] = model_xgb.predict(IV[X_name])

for Y_name in Y:
    save_path_dir = r'\%s_%s' %(Y_name, verson)
    if not os.path.exists(save_path_dir):
        os.makedirs(save_path_dir)
    IV2 = IV[IV[Y_name]>5]
    X_train, X_test, Y_train, Y_test = train_test_split(IV2.loc[:, X_name], IV2.loc[:, Y_name], train_size=0.99, shuffle=True, random_state=42)
    bayes_cv_tuner = BayesSearchCV(
                                    estimator = xgb.XGBRegressor(tree_method = 'gpu_hist', gpu_id = 2),
                                    search_spaces = {
                                                    'learning_rate': (0.05, 0.25, 'log-uniform'),
                                                    'min_child_weight': (1, 20, 'uniform'),
                                                    'max_depth': (5, 25, 'uniform'),
                                                    'max_delta_step': ( 0, 20, 'uniform'),
                                                    'alpha':(0, 5, 'uniform'),
                                                    'lambda':(0, 5, 'uniform'),
                                                    },    
                                    n_iter = 1000,   
                                    verbose = 3,
                                    cv = 5
                                    )
    bayes_cv_tuner.fit(X_train, Y_train)
    info = json.loads(json.dumps(bayes_cv_tuner.best_params_))
    X_train, X_test, Y_train, Y_test = train_test_split(IV2.loc[:, X_name], IV2.loc[:, Y_name], train_size=0.7, shuffle=True, random_state=42)
    eval_set = [(X_test, Y_test)]
    model_xgb = xgb.XGBRegressor(params=info, n_estimators = 1000)
    model_xgb.fit(X_train, Y_train, early_stopping_rounds = 100, eval_metric="rmse", eval_set=eval_set, verbose=False)
    X_train = IV[X_name]
    explainer = shap.explainers.GPUTree(model_xgb)
    shap_values = explainer.shap_values(X_train)
    shap_interaction_values = explainer.shap_interaction_values(X_train)

